In [1]:
# ---------------------------------------------------------------------
# Updated Colab cell: InceptionV3 two-stage fine-tune + robust evaluation
# - Prefers AdamW optimizer (with fallbacks)
# - Removes EarlyStopping so training runs all epochs
# - Adds training curves (loss/accuracy), confusion matrix, ROC, CSV of predictions
# - Saves metrics summary JSON and final model
# ---------------------------------------------------------------------

# ---------------------------
# Imports (top)
# ---------------------------
import os
import zipfile
import time
import math
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from collections import Counter

from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, roc_curve, auc, roc_auc_score
from sklearn.preprocessing import label_binarize

# ---------------------------
# Configuration (edit as needed)
# ---------------------------
DRIVE_MOUNT_POINT = "/content/drive"
ZIP_PATH = "/content/drive/MyDrive/d.zip"   # dataset zip on Drive (optional)
DATA_ROOT = "/content/dataset"
TRAIN_DIR = os.path.join(DATA_ROOT, "d", "train")
VAL_DIR   = os.path.join(DATA_ROOT, "d", "val")

OUT_DIR = "/content/drive/MyDrive/inceptionv3_results"
CHECKPOINT_PATH = os.path.join(OUT_DIR, "best_inceptionv3.keras")
FINAL_MODEL_PATH = os.path.join(OUT_DIR, "final_inceptionv3_model.keras")
CSV_PRED = os.path.join(OUT_DIR, "val_predictions_inceptionv3.csv")
METRICS_JSON = os.path.join(OUT_DIR, "training_summary_inceptionv3.json")

TARGET_SIZE = (299, 299)
BATCH_SIZE = 32
SEED = 42

# Training schedule: two stages
EPOCHS_HEAD = 8
EPOCHS_FINETUNE = 43   # total = 8 + 43 = 51
TOTAL_EPOCHS = EPOCHS_HEAD + EPOCHS_FINETUNE

# Choose optimizer: 'sgd', 'adamw', 'nadam', 'adamax'
OPTIMIZER_NAME = 'adamw'

# Optimizer hyperparams (defaults)
LR_HEAD = 1e-4
LR_FINETUNE = 1e-5
SGD_LR = 0.01
SGD_MOM = 0.9
ADAMW_WD = 1e-5
ADAMW_LR = LR_HEAD
NADAM_LR = 1e-4
ADAMAX_LR = 2e-3

# Regularization
APPLY_L2_TO_HEAD = True
L2_WEIGHT = 1e-4   # used for SGD/Nadam/Adamax to emulate weight-decay in the head

# Augmentation & mixup
USE_MIXUP = False
MIXUP_ALPHA = 0.2

# Mixed precision (optional)
USE_MIXED_PRECISION = False

# ---------------------------
# Mount Drive (Colab)
# ---------------------------
from google.colab import drive
_drive = drive.mount(DRIVE_MOUNT_POINT, force_remount=False)

# ---------------------------
# Repro & optional mixed precision
# ---------------------------
np.random.seed(SEED)
np.random.RandomState(SEED)
tf.random.set_seed(SEED)
os.environ['PYTHONHASHSEED'] = str(SEED)

if USE_MIXED_PRECISION:
    try:
        tf.keras.mixed_precision.set_global_policy('mixed_float16')
        print("Mixed precision enabled.")
    except Exception as e:
        print("Mixed precision not enabled:", e)

# ---------------------------
# Extract dataset if zipped (optional)
# ---------------------------
os.makedirs(DATA_ROOT, exist_ok=True)
if os.path.exists(ZIP_PATH) and not (os.path.exists(TRAIN_DIR) and os.path.exists(VAL_DIR)):
    print("Extracting dataset zip from Drive to", DATA_ROOT)
    with zipfile.ZipFile(ZIP_PATH, 'r') as z:
        z.extractall(DATA_ROOT)
    print("Extraction complete.")
else:
    print("Dataset already present or zip not found. DATA_ROOT =", DATA_ROOT)

# ---------------------------
# Compute class counts & class weights
# ---------------------------
def class_counts_from_dir(dirpath):
    counts = {}
    if not os.path.exists(dirpath):
        return counts
    for cls in sorted(os.listdir(dirpath)):
        cls_path = os.path.join(dirpath, cls)
        if os.path.isdir(cls_path):
            counts[cls] = len([f for f in os.listdir(cls_path) if os.path.isfile(os.path.join(cls_path, f))])
    return counts

train_counts = class_counts_from_dir(TRAIN_DIR)
val_counts   = class_counts_from_dir(VAL_DIR)
print("Train class counts:", train_counts)
print("Val   class counts:", val_counts)


def make_class_weights(counts):
    if not counts:
        return None
    labels = sorted(counts.keys())
    freqs = np.array([counts[k] for k in labels], dtype=np.float32)
    total = freqs.sum()
    weights = (total / (len(labels) * freqs))
    return dict(zip(labels, weights))

class_weights_by_name = make_class_weights(train_counts)
print("Per-class weights by name:", class_weights_by_name)

# ---------------------------
# Build tf.data datasets (deterministic order)
# ---------------------------
AUTOTUNE = tf.data.AUTOTUNE

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    TRAIN_DIR,
    labels="inferred",
    label_mode="int",
    batch_size=BATCH_SIZE,
    image_size=TARGET_SIZE,
    shuffle=True,
    seed=SEED
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    VAL_DIR,
    labels="inferred",
    label_mode="int",
    batch_size=BATCH_SIZE,
    image_size=TARGET_SIZE,
    shuffle=False
)

class_names = train_ds.class_names
NUM_CLASSES = len(class_names)
print("Detected classes (index->name):", dict(enumerate(class_names)), "NUM_CLASSES=", NUM_CLASSES)

# convert class_weights_by_name to index-based mapping
class_weights = None
if class_weights_by_name:
    class_weights = {i: float(class_weights_by_name[name]) for i, name in enumerate(class_names) if name in class_weights_by_name}
    print("Class weights (index):", class_weights)

# ---------------------------
# Preprocessing & augmentation
# ---------------------------
from tensorflow.keras.applications.inception_v3 import preprocess_input as inception_preprocess

data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip('horizontal'),
    tf.keras.layers.RandomRotation(0.12),
    tf.keras.layers.RandomZoom(0.12),
    tf.keras.layers.RandomContrast(0.08),
], name="data_augmentation")


def sample_beta_distribution(size, concentration=MIXUP_ALPHA):
    return np.random.beta(concentration, concentration, size)


def mixup(batch_x, batch_y, alpha=MIXUP_ALPHA):
    lam = sample_beta_distribution(batch_x.shape[0]).astype('float32')
    lam_x = lam.reshape(-1, 1, 1, 1)
    lam_y = lam.reshape(-1, 1)
    index = np.random.permutation(batch_x.shape[0])
    mixed_x = batch_x * lam_x + batch_x[index] * (1 - lam_x)
    y1 = tf.one_hot(batch_y, NUM_CLASSES)
    y2 = tf.one_hot(batch_y[index], NUM_CLASSES)
    mixed_y = y1 * lam_y + y2 * (1 - lam_y)
    return mixed_x, mixed_y


def prepare_train(ds, augment=True, mixup_prob=0.0):
    def _map(x, y):
        x = tf.cast(x, tf.float32)
        x = inception_preprocess(x)
        if augment:
            x = data_augmentation(x)
        return x, y
    ds = ds.map(_map, num_parallel_calls=AUTOTUNE)
    ds = ds.cache().prefetch(AUTOTUNE)
    if USE_MIXUP and mixup_prob > 0.0:
        def _mixup(x, y):
            x_m, y_m = mixup(x, y, alpha=mixup_prob)
            return x_m.astype('float32'), y_m.astype('float32')
        def _mixup_tf(x, y):
            x_m, y_m = tf.numpy_function(_mixup, [x, y], [tf.float32, tf.float32])
            x_m.set_shape([None, TARGET_SIZE[0], TARGET_SIZE[1], 3])
            y_m.set_shape([None, NUM_CLASSES])
            return x_m, y_m
        ds = ds.map(_mixup_tf, num_parallel_calls=AUTOTUNE)
    else:
        def _to_onehot(x, y):
            return x, tf.one_hot(y, NUM_CLASSES)
        ds = ds.map(_to_onehot, num_parallel_calls=AUTOTUNE)
    return ds


def prepare_val(ds):
    def _map(x, y):
        x = tf.cast(x, tf.float32)
        x = inception_preprocess(x)
        return x, y
    ds = ds.map(_map, num_parallel_calls=AUTOTUNE)
    ds = ds.cache().prefetch(AUTOTUNE)
    return ds

train_ds_prepared = prepare_train(train_ds, augment=True, mixup_prob=MIXUP_ALPHA if USE_MIXUP else 0.0)
val_ds_prepared   = prepare_val(val_ds)

# ---------------------------
# Model: InceptionV3 base + head
# ---------------------------
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.applications import InceptionV3


def build_model(input_shape=(*TARGET_SIZE, 3), n_classes=NUM_CLASSES, dropout_rate=0.5, l2_reg=None):
    base = InceptionV3(include_top=False, weights='imagenet', input_shape=input_shape, pooling='avg')
    x = base.output
    x = layers.BatchNormalization()(x)
    if l2_reg:
        x = layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(l2_reg))(x)
    else:
        x = layers.Dense(256, activation='relu')(x)
    x = layers.Dropout(dropout_rate)(x)
    x = layers.BatchNormalization()(x)
    if l2_reg:
        outputs = layers.Dense(n_classes, activation='softmax', kernel_regularizer=regularizers.l2(l2_reg), dtype='float32')(x)
    else:
        outputs = layers.Dense(n_classes, activation='softmax', dtype='float32')(x)
    model = models.Model(inputs=base.input, outputs=outputs)
    return model, base

l2_for_head = L2_WEIGHT if (APPLY_L2_TO_HEAD and OPTIMIZER_NAME.lower() in ['sgd','nadam','adamax']) else None
model, base_model = build_model(l2_reg=l2_for_head)
model.summary()

# Freeze early layers for stage 1 (freeze entire base for determinism)
for i, layer in enumerate(base_model.layers):
    layer.trainable = False

# ---------------------------
# Optimizer selection (with AdamW fallback)
# ---------------------------
def get_optimizer(name="adamw", lr=None, wd=ADAMW_WD):
    n = name.lower().strip()
    if n == "sgd":
        return tf.keras.optimizers.SGD(learning_rate=lr or SGD_LR, momentum=SGD_MOM, nesterov=True)
    if n == "nadam":
        return tf.keras.optimizers.Nadam(learning_rate=lr or NADAM_LR)
    if n == "adamax":
        return tf.keras.optimizers.Adamax(learning_rate=lr or ADAMAX_LR)
    if n == "adamw":
        # try tf.keras AdamW, else experimental, else tensorflow-addons
        try:
            return tf.keras.optimizers.AdamW(learning_rate=lr or ADAMW_LR, weight_decay=wd)
        except Exception:
            try:
                return tf.keras.optimizers.experimental.AdamW(learning_rate=lr or ADAMW_LR, weight_decay=wd)
            except Exception:
                try:
                    import tensorflow_addons as tfa
                    return tfa.optimizers.AdamW(learning_rate=lr or ADAMW_LR, weight_decay=wd)
                except Exception:
                    print("AdamW not available. Falling back to Adamax.")
                    return tf.keras.optimizers.Adamax(learning_rate=lr or ADAMAX_LR)
    raise ValueError("Unsupported optimizer: " + name)

# Print optimizer choice
print("Using optimizer choice (name):", OPTIMIZER_NAME)

# ---------------------------
# Callbacks (NO EarlyStopping)
# ---------------------------
os.makedirs(OUT_DIR, exist_ok=True)
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(
    CHECKPOINT_PATH, monitor='val_loss', save_best_only=True, save_weights_only=False, verbose=1
)
reduce_lr_cb = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=3, min_lr=1e-7, verbose=1)
# EarlyStopping intentionally removed so training completes all epochs
tensorboard_cb = tf.keras.callbacks.TensorBoard(log_dir=os.path.join(OUT_DIR, "tb_logs"), histogram_freq=1)

# ---------------------------
# Stage 1: train head only (base frozen)
# ---------------------------
opt_stage1 = get_optimizer(OPTIMIZER_NAME, lr=LR_HEAD)
model.compile(optimizer=opt_stage1, loss='categorical_crossentropy', metrics=['accuracy'])

print("\n=== Stage 1: Training head (base frozen) ===")
history_head = model.fit(
    train_ds_prepared,
    validation_data=val_ds_prepared.map(lambda x,y:(x, tf.one_hot(y, NUM_CLASSES))),
    epochs=EPOCHS_HEAD,
    callbacks=[checkpoint_cb, reduce_lr_cb, tensorboard_cb],
    class_weight=class_weights,
    verbose=1
)

# ---------------------------
# Stage 2: fine-tune (unfreeze & small LR)
# ---------------------------
for layer in base_model.layers:
    layer.trainable = True
# keep BatchNorm layers frozen for stability
for layer in base_model.layers:
    if isinstance(layer, tf.keras.layers.BatchNormalization):
        layer.trainable = False

opt_stage2 = get_optimizer(OPTIMIZER_NAME, lr=LR_FINETUNE)
model.compile(optimizer=opt_stage2, loss='categorical_crossentropy', metrics=['accuracy'])

print("\n=== Stage 2: Fine-tuning (base unfrozen) ===")
history_ft = model.fit(
    train_ds_prepared,
    validation_data=val_ds_prepared.map(lambda x,y:(x, tf.one_hot(y, NUM_CLASSES))),
    epochs=TOTAL_EPOCHS,
    initial_epoch=EPOCHS_HEAD,
    callbacks=[checkpoint_cb, reduce_lr_cb, tensorboard_cb],
    class_weight=class_weights,
    verbose=1
)

# ---------------------------
# Combine histories & plot training curves
# ---------------------------

def combine_histories(h1, h2):
    out = {}
    for k in h1.history.keys():
        out[k] = h1.history[k] + h2.history.get(k, [])
    for k in h2.history.keys():
        if k not in out:
            out[k] = h2.history[k]
    return out

combined = combine_histories(history_head, history_ft)


def plot_and_save_training_curves(history_dict, out_dir):
    epochs = range(1, len(history_dict.get('loss', [])) + 1)
    # Accuracy
    if 'accuracy' in history_dict:
        plt.figure(figsize=(8,6))
        plt.plot(epochs, history_dict['accuracy'], label='train_acc')
        if 'val_accuracy' in history_dict:
            plt.plot(epochs, history_dict['val_accuracy'], label='val_acc')
        plt.xlabel('Epoch'); plt.ylabel('Accuracy'); plt.title('Training & Validation Accuracy')
        plt.legend(); plt.grid(True)
        p = os.path.join(out_dir, "accuracy_curve.png")
        plt.savefig(p); plt.close()
        print("Saved accuracy plot to:", p)
    # Loss
    if 'loss' in history_dict:
        plt.figure(figsize=(8,6))
        plt.plot(epochs, history_dict['loss'], label='train_loss')
        if 'val_loss' in history_dict:
            plt.plot(epochs, history_dict['val_loss'], label='val_loss')
        plt.xlabel('Epoch'); plt.ylabel('Loss'); plt.title('Training & Validation Loss')
        plt.legend(); plt.grid(True)
        p = os.path.join(out_dir, "loss_curve.png")
        plt.savefig(p); plt.close()
        print("Saved loss plot to:", p)

plot_and_save_training_curves(combined, OUT_DIR)

# ---------------------------
# Save final model
# ---------------------------
try:
    model.save(FINAL_MODEL_PATH)
    print("Saved final model to:", FINAL_MODEL_PATH)
except Exception as e:
    print("Saving .keras failed, attempting .h5 fallback:", e)
    try:
        fallback = FINAL_MODEL_PATH.replace(".keras", ".h5")
        model.save(fallback)
        print("Saved fallback model to:", fallback)
    except Exception as e2:
        print("Final save also failed:", e2)

# ---------------------------
# Load best checkpoint if present
# ---------------------------
best_model_path = CHECKPOINT_PATH if os.path.exists(CHECKPOINT_PATH) else FINAL_MODEL_PATH
print("Attempting to load best model from:", best_model_path)
try:
    model = tf.keras.models.load_model(best_model_path)
    print("Loaded model:", best_model_path)
except Exception as e:
    print("Could not load checkpoint; continuing with in-memory model. Error:", e)

# ---------------------------
# Robust evaluation: predictions, confusion matrix, ROC, CSV
# ---------------------------
# Reconstruct validation file list in deterministic order class/filename
val_filepaths = []
for cls in class_names:
    cls_dir = os.path.join(VAL_DIR, cls)
    if not os.path.isdir(cls_dir):
        continue
    files = sorted([f for f in os.listdir(cls_dir) if os.path.isfile(os.path.join(cls_dir, f))])
    for fname in files:
        val_filepaths.append(os.path.join(cls, fname))
print(f"Reconstructed {len(val_filepaths)} validation file paths.")

# Build prediction dataset (shuffle=False)
val_pred_ds = tf.keras.preprocessing.image_dataset_from_directory(
    VAL_DIR,
    labels="inferred",
    label_mode="int",
    batch_size=BATCH_SIZE,
    image_size=TARGET_SIZE,
    shuffle=False
)
val_pred_ds_pp = val_pred_ds.map(lambda x,y: (inception_preprocess(tf.cast(x, tf.float32)), y)).prefetch(AUTOTUNE)

# Predict
pred_probs = model.predict(val_pred_ds_pp, verbose=1)

# Extract y_true robustly
y_true = np.concatenate([y.numpy() for x, y in val_pred_ds], axis=0)
y_pred = np.argmax(pred_probs, axis=1)

# Sanity check
if pred_probs.shape[0] != len(y_true):
    print("WARNING: number of predictions does not match number of true labels.")
    print("pred_probs rows:", pred_probs.shape[0], "y_true len:", len(y_true), "reconstructed filenames:", len(val_filepaths))

# Map index->label
index_to_label = {i: name for i, name in enumerate(class_names)}
target_names = [index_to_label[i] for i in sorted(index_to_label.keys())]

# Accuracy & confusion matrix
acc = accuracy_score(y_true, y_pred) if len(y_true) > 0 else 0.0
print(f"\nValidation accuracy: {acc*100:.2f}%")

cm = confusion_matrix(y_true, y_pred, labels=sorted(index_to_label.keys()))
plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=target_names, yticklabels=target_names)
plt.xlabel("Predicted"); plt.ylabel("Actual"); plt.title("Confusion Matrix")
cm_path = os.path.join(OUT_DIR, "confusion_matrix.png")
plt.savefig(cm_path)
plt.close()
print("Saved confusion matrix to:", cm_path)

print("\nClassification report:\n")
print(classification_report(y_true, y_pred, target_names=target_names, zero_division=0))

# ROC per-class (one-vs-rest)
roc_path = os.path.join(OUT_DIR, "roc_curve.png")
try:
    y_true_bin = label_binarize(y_true, classes=list(range(NUM_CLASSES)))
    fpr, tpr, roc_auc = {}, {}, {}
    present_classes = []
    for i in range(NUM_CLASSES):
        if np.sum(y_true_bin[:, i]) == 0:
            print(f"Skipping ROC for class {i} ({index_to_label.get(i,'?')}) — no positives in y_true")
            continue
        fpr[i], tpr[i], _ = roc_curve(y_true_bin[:, i], pred_probs[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])
        present_classes.append(i)

    plt.figure(figsize=(9,8))
    for i in present_classes:
        plt.plot(fpr[i], tpr[i], label=f"{index_to_label.get(i, i)} (AUC={roc_auc[i]:.3f})")
    plt.plot([0,1],[0,1],'k--', label='Chance')
    plt.xlabel("False Positive Rate"); plt.ylabel("True Positive Rate"); plt.title("ROC Curves (one-vs-rest)")
    plt.legend(loc='lower right'); plt.grid(True)
    plt.savefig(roc_path)
    plt.close()
    print("Saved ROC plot to:", roc_path)
except Exception as e:
    print("ROC plotting skipped due to:", e)

# For binary classification, also compute single AUC using positive class probs (index 1)
auc_score = None
try:
    if NUM_CLASSES == 2 and pred_probs.shape[1] >= 2 and len(y_true) > 0:
        auc_score = roc_auc_score(y_true, pred_probs[:, 1])
        print(f"Binary ROC AUC (class '{index_to_label.get(1,1)}' as positive): {auc_score:.4f}")
except Exception as e:
    print("Binary AUC computation failed:", e)

# Save CSV of predictions
rows = []
min_len = min(len(val_filepaths), len(y_true), pred_probs.shape[0])
if min_len < len(y_true):
    print("NOTE: filename count differs from true/pred count — using min_len =", min_len)

for i in range(min_len):
    filename = val_filepaths[i] if i < len(val_filepaths) else f"unknown_{i}"
    rows.append({
        "filename": filename,
        "true_label": index_to_label[int(y_true[i])],
        "pred_label": index_to_label[int(y_pred[i])],
        **{f"prob_{index_to_label[j]}": float(pred_probs[i, j]) for j in range(pred_probs.shape[1])}
    })

if len(y_true) > min_len:
    for i in range(min_len, len(y_true)):
        rows.append({
            "filename": f"unknown_{i}",
            "true_label": index_to_label[int(y_true[i])],
            "pred_label": index_to_label[int(y_pred[i])],
            **{f"prob_{index_to_label[j]}": float(pred_probs[i, j]) for j in range(pred_probs.shape[1])}
        })


df = pd.DataFrame(rows)
df.to_csv(CSV_PRED, index=False)
print("Saved per-sample predictions to:", CSV_PRED)

# ---------------------------
# Save summary metrics & training info
# ---------------------------
metrics_summary = {
    "num_classes": int(NUM_CLASSES),
    "class_names": class_names,
    "train_counts": train_counts,
    "val_counts": val_counts,
    "final_validation_accuracy": float(acc) if 'acc' in locals() else None,
    "binary_auc_positive_class_1": float(auc_score) if auc_score is not None else None,
    "epochs_head": int(EPOCHS_HEAD),
    "epochs_finetune": int(EPOCHS_FINETUNE),
    "total_epochs": int(TOTAL_EPOCHS),
    "optimizer": OPTIMIZER_NAME,
    "learning_rate_head": LR_HEAD,
    "learning_rate_finetune": LR_FINETUNE,
    "class_weights": class_weights,
    "artifacts": {
        "checkpoint": best_model_path,
        "final_model": FINAL_MODEL_PATH,
        "predictions_csv": CSV_PRED,
        "confusion_matrix": cm_path,
        "roc_curve": roc_path,
        "accuracy_curve": os.path.join(OUT_DIR, "accuracy_curve.png"),
        "loss_curve": os.path.join(OUT_DIR, "loss_curve.png")
    }
}

with open(METRICS_JSON, "w") as f:
    json.dump(metrics_summary, f, indent=2)
print("Saved training summary JSON to:", METRICS_JSON)

print("\nAll evaluation artifacts saved in:", OUT_DIR)


Mounted at /content/drive
Extracting dataset zip from Drive to /content/dataset
Extraction complete.
Train class counts: {'benign': 1440, 'malignant': 1197}
Val   class counts: {'benign': 360, 'malignant': 300}
Per-class weights by name: {'benign': np.float32(0.915625), 'malignant': np.float32(1.1015037)}
Found 2637 files belonging to 2 classes.
Found 660 files belonging to 2 classes.
Detected classes (index->name): {0: 'benign', 1: 'malignant'} NUM_CLASSES= 2
Class weights (index): {0: 0.9156249761581421, 1: 1.1015037298202515}
87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 299, 299,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 149, 149,  │        864 │ input_layer_1[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 149, 149,  │         96 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 149, 149,  │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 147, 147,  │      9,216 │ activation[0][0]  │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 147, 147,  │         96 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 147, 147,  │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 147, 147,  │     18,432 │ activation_1[0][… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 147, 147,  │        192 │ conv2d_2[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_2        │ (None, 147, 147,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 73, 73,    │          0 │ activation_2[0][… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 73, 73,    │      5,120 │ max_pooling2d[0]… │
│                     │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 73, 73,    │        240 │ conv2d_3[0][0]    │
│ (BatchNormalizatio… │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_3        │ (None, 73, 73,    │          0 │ batch_normalizat… │
│ (Activation)        │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 71, 71,    │    138,240 │ activation_3[0][… │
│                     │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 71, 71,    │        576 │ conv2d_4[0][0]    │
│ (BatchNormalizatio… │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_4        │ (None, 71, 71,    │          0 │ batch_normalizat

 Total params: 22,337,058 (85.21 MB)

 Trainable params: 22,298,018 (85.06 MB)

 Non-trainable params: 39,040 (152.50 KB)

Using optimizer choice (name): adamw

=== Stage 1: Training head (base frozen) ===
Epoch 1/8
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 705ms/step - accuracy: 0.6476 - loss: 0.7852
Epoch 1: val_loss improved from inf to 0.51540, saving model to /content/drive/MyDrive/inceptionv3_results/best_inceptionv3.keras
83/83 ━━━━━━━━━━━━━━━━━━━━ 114s 1s/step - accuracy: 0.6484 - loss: 0.7838 - val_accuracy: 0.7273 - val_loss: 0.5154 - learning_rate: 1.0000e-04
Epoch 2/8
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step - accuracy: 0.7955 - loss: 0.4592
Epoch 2: val_loss improved from 0.51540 to 0.46229, saving model to /content/drive/MyDrive/inceptionv3_results/best_inceptionv3.keras
83/83 ━━━━━━━━━━━━━━━━━━━━ 19s 234ms/step - accuracy: 0.7955 - loss: 0.4593 - val_accuracy: 0.7561 - val_loss: 0.4623 - learning_rate: 1.0000e-04
Epoch 3/8
83/83 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step - accuracy: 0.8331 - loss: 0.3889
Epoch 3: val_loss did not improve from 0.46229
83/83 ━━━━━━━━━━━━━━━━━━━━ 20s 239ms/step - accuracy: 0.8331 